In [1]:
import pandas as pd
import numpy as np

In [2]:
main_df = pd.read_csv('../../../Project/data/Main.csv') # update to trimmed dataset later on and remove the next few cells

C:\Users\audre\AppData\Local\Temp\ipykernel_23056\2025641769.py:1: DtypeWarning: Columns (4,11,14,18,19,23,25,32,34,35,36,37,38,39,40,41,46,57,59,64,65,66,67,68,71,74,79,80,81,82,83,84,85,86,89,90,91,95,96,106,109,110,111,112,113,130,133,136,137,141,142,143,144,338,339,340,342,344,345,346,347,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,372,381,382,383,384,385,386,387,388,389,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,416,417,418,419,420,421,423,424,528,529,535,536,540,541,542,543,544) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv('../../../Project/data/Main.csv') # update to trimmed dataset later on and remove the next few cells


In [7]:
main_df.rename(columns=str.upper, inplace=True)

In [8]:
main_df.head()

,CID,DOCKET,OFN_DOCKET,JPR_ID,OFF_PPID,OFF_SEX,OFF_RACE,OFN_ID,DOFAGE,OTN,...,RIP5_MIN_DAYS,RIP5_MIN_MONTHS,RIPMIN5,RIP_TYPE5,OFN_LABEL,OFN_PCSSUBCODE,JUDGE_LNAME,MND_DESCRIPTION,PRSADJ_139,PRSCONV_139
0,CID_1102800013N,DK_2149-2000,NaN,640001,NaN,F,White,869,36.689938,H182628-5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CID_52695N,DK_2098-00,NaN,642480,NaN,M,White,934,18.540726,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CID_314158N,DK_0932/01,NaN,660434,NaN,M,White,934,36.914442,H3618344,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CID_200602445N,DK_200014898,NaN,628940,279104,M,Black,1023,22.297057,G0816126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CID_1600900655N,DK_816CR2000,NaN,594048,NaN,M,White,916,40.087611,H240127-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### County info

In [9]:
main_df['COUNTY'].unique()

array(['Cambria', 'Chester', 'Bucks', 'Allegheny', 'Clarion',
       'Westmoreland', 'Lehigh', 'Philadelphia', 'Delaware', 'Montgomery',
       'Crawford', 'Washington', 'Cumberland', 'Franklin', 'Lackawanna',
       'Erie', 'Berks', 'Dauphin', 'York', 'Fayette', 'Lancaster',
       'Luzerne', 'Schuylkill', 'Mercer', 'Beaver', 'Armstrong',
       'Venango', 'Monroe', 'Lebanon', 'Carbon', 'Northampton',
       'Northumberland', 'Bradford', 'Union', 'Mifflin', 'Wyoming',
       'Centre', 'Susquehanna', 'Lawrence', 'Greene', 'Blair', 'Juniata',
       'Lycoming', 'Adams', 'Columbia', 'Butler', 'McKean', 'Forest',
       'Snyder', 'Indiana', 'Clearfield', 'Perry', 'Wayne', 'Huntingdon',
       'Tioga', 'Somerset', 'Clinton', 'Elk', 'Warren', 'Pike', 'Fulton',
       'Montour', 'Potter', 'Bedford', 'Sullivan', 'Jefferson', 'Cameron',
       'PCCD'], dtype=object)

In [10]:
main_df['CTY_PHL'] = np.where(main_df['COUNTY']=='Philadelphia', 1, 0)

In [12]:
main_df['DOS'] = pd.to_datetime(main_df['DOS'])

In [27]:
# cases sentenced in different counties on the same date 
main_df.groupby(by=['ID_VARIABLE', 'DOS'])['COUNTY'].nunique().sort_values(ascending=False)

ID_VARIABLE  DOS       
1017202      2009-01-08    3
1162952      2009-04-07    2
1617687      2018-04-19    2
1815707      2007-12-10    2
1490773      2015-10-14    2
                          ..
1305653      2011-05-02    1
1305652      2007-09-05    1
             2007-07-25    1
1305651      2012-12-10    1
1916197      2018-08-30    1
Name: COUNTY, Length: 1496315, dtype: int64

In [23]:
main_df[(main_df['ID_VARIABLE']==1413417) & (main_df['DOS'] == pd.to_datetime('2012-06-08'))]['COUNTY'].unique()

array(['Philadelphia', 'Bucks'], dtype=object)

In [278]:
# we will count it as a Philly case if at least one county is Philly, by taking max(CTY_PHL) in the aggregation
phl = main_df.groupby(by=['ID_VARIABLE', 'JPR_ID']).agg({'CTY_PHL': 'max', 'DOS': 'max'})

In [279]:
phl = phl.reset_index().groupby(by=['ID_VARIABLE', 'DOS'])['CTY_PHL'].max().reset_index()

In [319]:
print(len(phl))

1495982


### Number of charges

In [281]:
# check cases where one judicial proceeding was heard multiple times (e.g. multiple DOS per JP)
main_df.groupby(by=['ID_VARIABLE', 'JPR_ID'])['DOS'].nunique().sort_values(ascending=False)

ID_VARIABLE  JPR_ID 
1884187      5199873    3
1692377      5087069    3
1819935      5299118    3
1528197      5051245    2
1130813      5350926    2
                       ..
1305594      271993     1
             271930     1
1305593      5431263    1
             2864596    1
1916197      5830496    1
Name: DOS, Length: 1581700, dtype: int64

In [282]:
# for cases with multiple sentencing dates per JP, the charges are represented multiple times for each time the case was adjudicated on each DOS
main_df[(main_df['ID_VARIABLE']==1884187) & (main_df['JPR_ID']==5199873)][['ID_VARIABLE', 'DOS', 'OFN_LABEL', 'JPR_ID', 'OFN_ID', 'INC_RELATIONSHIP']].sort_values(by=['DOS', 'OFN_ID'])

,ID_VARIABLE,DOS,OFN_LABEL,OFN_LABEL,JPR_ID,OFN_ID,INC_RELATIONSHIP
1591799,1884187,2012-12-06,Possession With Intent to Deliver: Cocaine (10...,NaN,5199873,214,NaN
1591802,1884187,2012-12-06,Possession With Intent to Deliver: Cocaine (10...,NaN,5199873,214,NaN
1591800,1884187,2012-12-06,Simple Possession - 2nd/Subsequent Offense,NaN,5199873,327,NaN
1591803,1884187,2012-12-06,Possession of Drug Paraphernalia,NaN,5199873,357,NaN
1591801,1884187,2012-12-06,Firearms-Persons Not To Possess - Convicted of...,NaN,5199873,9341,Concurrent
1591804,1884187,2012-12-06,Firearms-Persons Not To Possess - Convicted of...,NaN,5199873,9341,Consecutive
1885847,1884187,2014-10-09,Possession With Intent to Deliver: Cocaine (10...,NaN,5199873,214,NaN
1885851,1884187,2014-10-09,Possession With Intent to Deliver: Cocaine (10...,NaN,5199873,214,NaN
1885849,1884187,2014-10-09,Simple Possession - 2nd/Subsequent Offense,NaN,5199873,327,NaN
1885850,1884187,2014-10-09,Possession of Drug Paraphernalia,NaN,5199873,357,NaN


In [283]:
main_df[(main_df['ID_VARIABLE']==1884187) & (main_df['JPR_ID']==5199873)][['ID_VARIABLE', 'DOS', 'OFN_LABEL', 'JPR_ID', 'OFN_ID', 'INCMIN', 'JP_MIN', 'INC_RELATIONSHIP']].sort_values(by=['DOS', 'OFN_ID'])

,ID_VARIABLE,DOS,OFN_LABEL,OFN_LABEL,JPR_ID,OFN_ID,INCMIN,JP_MIN,INC_RELATIONSHIP
1591799,1884187,2012-12-06,Possession With Intent to Deliver: Cocaine (10...,NaN,5199873,214,NaN,2556.0,NaN
1591802,1884187,2012-12-06,Possession With Intent to Deliver: Cocaine (10...,NaN,5199873,214,60.0,2556.0,NaN
1591800,1884187,2012-12-06,Simple Possession - 2nd/Subsequent Offense,NaN,5199873,327,NaN,2556.0,NaN
1591803,1884187,2012-12-06,Possession of Drug Paraphernalia,NaN,5199873,357,NaN,2556.0,NaN
1591801,1884187,2012-12-06,Firearms-Persons Not To Possess - Convicted of...,NaN,5199873,9341,24.0,2556.0,Concurrent
1591804,1884187,2012-12-06,Firearms-Persons Not To Possess - Convicted of...,NaN,5199873,9341,24.0,2556.0,Consecutive
1885847,1884187,2014-10-09,Possession With Intent to Deliver: Cocaine (10...,NaN,5199873,214,60.0,2557.0,NaN
1885851,1884187,2014-10-09,Possession With Intent to Deliver: Cocaine (10...,NaN,5199873,214,NaN,2557.0,NaN
1885849,1884187,2014-10-09,Simple Possession - 2nd/Subsequent Offense,NaN,5199873,327,NaN,2557.0,NaN
1885850,1884187,2014-10-09,Possession of Drug Paraphernalia,NaN,5199873,357,NaN,2557.0,NaN


In [284]:
# for multiple charges, we also need concurrent/consecutive sentence info
# for now, if there are both concurrent and consecutive sentences, we will flag it as concurrent. !!! double check with clients
order = ['Consecutive', 'Concurrent']

In [285]:
main_df['INC_REL_NUMERIC'] = main_df['INC_RELATIONSHIP'].map(dict(zip(order, np.arange(len(order)))))

In [304]:
# we only want to count the number of charges associated with the first DOS for each JPR_ID (i.e. the number of charges the first time that the case was sentenced, as well as the associated inc relationship)
charge_counts = main_df.groupby(by=['ID_VARIABLE', 'JPR_ID', 'DOS']).agg({'OFN_ID': 'count',
                                                                        'OGS': 'max',
                                                                        'INC_REL_NUMERIC': 'max', 
                                                                        'JP_MIN': 'max'
                                                                        }).reset_index()

In [305]:
charge_counts.rename(columns={'OFN_ID': 'CHARGE_COUNT'}, inplace=True)

In [306]:
# first create NEW_DOS column to use the latest DOS per JP as final DOS info
charge_counts['NEW_DOS'] = charge_counts.groupby(by=['ID_VARIABLE', 'JPR_ID'])['DOS'].transform('max')

In [307]:
# next group by JPR_ID and get the charge counts associated with the first DOS per JP 
charge_counts = charge_counts.loc[charge_counts.groupby(['ID_VARIABLE', 'JPR_ID'])['DOS'].idxmin()]

In [308]:
charge_counts[charge_counts['ID_VARIABLE']==1884187]

,ID_VARIABLE,JPR_ID,DOS,CHARGE_COUNT,OGS,INC_REL_NUMERIC,JP_MIN,NEW_DOS
1527095,1884187,1951956,2007-08-16,2,3.0,NaN,365.0,2007-08-16
1527096,1884187,5126496,2012-08-23,1,1.0,NaN,92.0,2012-08-23
1527097,1884187,5199873,2012-12-06,6,9.0,1.0,2556.0,2015-05-14


In [309]:
charge_counts[charge_counts['ID_VARIABLE']==1001234]

,ID_VARIABLE,JPR_ID,DOS,CHARGE_COUNT,OGS,INC_REL_NUMERIC,JP_MIN,NEW_DOS
2082,1001234,621649,2001-10-24,3,5.0,NaN,NaN,2001-10-24
2083,1001234,662988,2001-06-15,3,3.0,NaN,NaN,2001-06-15
2084,1001234,1137617,2008-04-14,2,7.0,1.0,365.0,2008-04-14
2085,1001234,2559776,2008-04-14,1,7.0,NaN,365.0,2008-04-14
2086,1001234,2652233,2008-04-14,1,8.0,NaN,1096.0,2008-04-14
2087,1001234,2665980,2008-02-28,3,8.0,1.0,3653.0,2008-02-28
2088,1001234,2734838,2008-05-06,2,4.0,NaN,350.0,2008-05-06


In [310]:
# we will use NEW_DOS from now on
charge_counts.rename(columns={'DOS': 'OLD_DOS', 'NEW_DOS': 'DOS'}, inplace=True)

In [311]:
charge_counts.head()

,ID_VARIABLE,JPR_ID,OLD_DOS,CHARGE_COUNT,OGS,INC_REL_NUMERIC,JP_MIN,DOS
0,1000001,4915383,2010-02-18,3,3.0,NaN,16.0,2010-02-18
1,1000002,5678165,2017-01-31,2,3.0,1.0,120.0,2017-01-31
2,1000003,1070248,2002-05-08,1,3.0,NaN,0.0,2002-05-08
3,1000003,4902797,2009-03-04,1,3.0,NaN,92.0,2009-03-04
4,1000004,5318013,2013-12-10,1,1.0,NaN,0.0,2013-12-10


In [312]:
# however, there are still people with multiple JPR_IDs on a given DOS that we need to take into consideration
charge_counts.groupby(by=['ID_VARIABLE', 'DOS'])['JPR_ID'].nunique().sort_values(ascending=False)

ID_VARIABLE  DOS       
1897774      2014-10-28    30
1470599      2003-10-31    26
1074875      2007-01-23    25
1899373      2018-01-11    25
1216661      2006-04-19    24
                           ..
1312335      2008-07-25     1
             2007-09-28     1
             2002-08-13     1
1312334      2009-04-02     1
1916197      2018-08-30     1
Name: JPR_ID, Length: 1495982, dtype: int64

In [313]:
# take a look at one example
main_df[(main_df['ID_VARIABLE']==1470599) & (main_df['DOS'] == pd.to_datetime('2003-10-31'))][['ID_VARIABLE', 'JPR_ID', 'DOS', 'OFN_LABEL', 'OGS', 'INC_RELATIONSHIP', 'JP_MIN', 'INCMIN']].sort_values(by='JPR_ID')

,ID_VARIABLE,JPR_ID,DOS,OFN_LABEL,OFN_LABEL,OGS,INC_RELATIONSHIP,JP_MIN,INCMIN
305024,1470599,18103,2003-10-31,"Theft - Receiving Stolen Property; > $2,000 - ...",NaN,5.0,NaN,1096.0,36.0
305038,1470599,18104,2003-10-31,Theft - Receiving Stolen Property; < $50,NaN,1.0,NaN,1095.0,NaN
305035,1470599,18104,2003-10-31,Theft - Receiving Stolen Property; < $50,NaN,1.0,NaN,1095.0,NaN
305033,1470599,18104,2003-10-31,"Theft - Receiving Stolen Property; > $2,000 - ...",NaN,5.0,NaN,1095.0,18.0
305016,1470599,18104,2003-10-31,"Theft - Receiving Stolen Property; > $2,000 - ...",NaN,5.0,Consecutive,1095.0,18.0
305042,1470599,18107,2003-10-31,Theft - Receiving Stolen Property; < $50,NaN,1.0,NaN,1096.0,NaN
305014,1470599,18107,2003-10-31,"Theft - Receiving Stolen Property; > $2,000 - ...",NaN,5.0,Concurrent,1096.0,36.0
305026,1470599,18107,2003-10-31,"Theft - Receiving Stolen Property; > $2,000 - ...",NaN,5.0,NaN,1096.0,36.0
305005,1470599,18109,2003-10-31,Burglary - Not of a Home: No One Present,NaN,5.0,NaN,1096.0,36.0
305041,1470599,18109,2003-10-31,"Theft - Unlawful Taking; > $2,000 - $25,000/Fi...",NaN,5.0,Concurrent,1096.0,36.0


In [314]:
charge_counts[(charge_counts['ID_VARIABLE']==1470599) & (charge_counts['DOS'] == pd.to_datetime('2003-10-31'))].sort_values(by='JPR_ID')

,ID_VARIABLE,JPR_ID,OLD_DOS,CHARGE_COUNT,OGS,INC_REL_NUMERIC,JP_MIN,DOS
812456,1470599,18103,2003-10-31,1,5.0,NaN,1096.0,2003-10-31
812457,1470599,18104,2003-10-31,4,5.0,0.0,1095.0,2003-10-31
812458,1470599,18107,2003-10-31,3,5.0,1.0,1096.0,2003-10-31
812459,1470599,18109,2003-10-31,3,5.0,1.0,1096.0,2003-10-31
812460,1470599,18111,2003-10-31,1,5.0,NaN,1096.0,2003-10-31
812461,1470599,18115,2003-10-31,1,5.0,NaN,1096.0,2003-10-31
812462,1470599,18116,2003-10-31,1,1.0,NaN,0.0,2003-10-31
812463,1470599,18118,2003-10-31,2,5.0,NaN,1096.0,2003-10-31
812464,1470599,18120,2003-10-31,1,5.0,NaN,1096.0,2003-10-31
812465,1470599,18122,2003-10-31,1,5.0,NaN,1096.0,2003-10-31


In [315]:
# FOR NOW: we will take the charge count associated with most severe sentence (largest JP_MIN) and highest OGS for a given DOS, after dealing with NANs
# TEAM: review later when incorporating into pipeline as to whether this logic makes sense 
charge_counts['OGS'] = charge_counts['OGS'].fillna(0)
charge_counts = charge_counts.sort_values(by=['ID_VARIABLE', 'DOS', 'JP_MIN', 'OGS', 'CHARGE_COUNT'], ascending=[True, True, False, False, False])

In [316]:
charge_counts_grouped = charge_counts.groupby(by=['ID_VARIABLE', 'DOS']).nth(0, dropna=False).reset_index()

In [317]:
# double check that this worked
charge_counts_grouped[(charge_counts_grouped['ID_VARIABLE']==1470599) & (charge_counts_grouped['DOS'] == pd.to_datetime('2003-10-31'))]

,ID_VARIABLE,DOS,JPR_ID,OLD_DOS,CHARGE_COUNT,OGS,INC_REL_NUMERIC,JP_MIN
768265,1470599,2003-10-31,18155,2003-10-31,5,5.0,1.0,1096.0


In [318]:
print(len(charge_counts_grouped))

1495982


##### Additional notes: Concurrent vs. consecutive for counts > 1 investigation

In [86]:
main_df['INC_RELATIONSHIP'].unique()

array([nan, 'Consecutive', 'Concurrent', 'X'], dtype=object)

In [87]:
multiple_sent = main_df[main_df['INC_RELATIONSHIP'].isin(['Concurrent', 'Consecutive'])]

In [88]:
# check that within each JP, the INC_RELATIONSHIP is the same. Sadly, that is not the case
multiple_sent.groupby(by=['ID_VARIABLE', 'JPR_ID'])['INC_RELATIONSHIP'].nunique().sort_values(ascending=False)

ID_VARIABLE  JPR_ID 
1533649      2732667    2
1435009      5258269    2
1321391      5396496    2
1843933      5852581    2
1874103      5838975    2
                       ..
1318351      29452      1
1318362      5010465    1
1318367      5933990    1
             6024830    1
1916184      66157      1
Name: INC_RELATIONSHIP, Length: 164303, dtype: int64

In [264]:
(multiple_sent.groupby(by=['ID_VARIABLE', 'JPR_ID'])['INC_RELATIONSHIP'].nunique().sort_values(ascending=False)>1).sum()

11349

In [100]:
main_df[(main_df['ID_VARIABLE']==1533649) & (main_df['JPR_ID']==2732667)][['ID_VARIABLE', 'JPR_ID', 'DOS', 'INC_RELATIONSHIP', 'INCMIN', 'JP_MIN', 'INC_TYPE', 'OFN_LABEL', 'OGS', 'MS_OFFINJP']]

,ID_VARIABLE,JPR_ID,DOS,INC_RELATIONSHIP,INCMIN,JP_MIN,INC_TYPE,OFN_LABEL,OFN_LABEL,OGS,MS_OFFINJP
939973,1533649,2732667,2008-05-08,NaN,6.0,365.0,State Facility,Terroristic Threats,NaN,3.0,Yes
939974,1533649,2732667,2008-05-08,Consecutive,6.0,365.0,State Facility,Terroristic Threats,NaN,3.0,NaN
939975,1533649,2732667,2008-05-08,Concurrent,1.0,365.0,State Facility,Theft - Receiving Stolen Property; $50 - < $200,NaN,2.0,NaN


In [101]:
main_df[(main_df['ID_VARIABLE']==1435009) & (main_df['JPR_ID']==5258269)][['ID_VARIABLE', 'JPR_ID', 'DOS', 'INC_RELATIONSHIP', 'INCMIN', 'JP_MIN', 'INC_TYPE', 'OFN_LABEL', 'OGS', 'MS_OFFINJP']]

,ID_VARIABLE,JPR_ID,DOS,INC_RELATIONSHIP,INCMIN,JP_MIN,INC_TYPE,OFN_LABEL,OFN_LABEL,OGS,MS_OFFINJP
1663878,1435009,5258269,2013-05-06,Consecutive,12.0,1461.0,State Facility,Possession With Intent to Deliver: Cocaine (< ...,NaN,6.0,NaN
1663879,1435009,5258269,2013-05-06,NaN,12.0,1461.0,State Facility,Possession With Intent to Deliver: Cocaine (< ...,NaN,6.0,NaN
1663880,1435009,5258269,2013-05-06,Concurrent,12.0,1461.0,State Facility,Possession With Intent to Deliver: Cocaine (< ...,NaN,6.0,NaN
1663881,1435009,5258269,2013-05-06,Consecutive,12.0,1461.0,State Facility,Possession With Intent to Deliver: Cocaine (< ...,NaN,6.0,Yes
1663883,1435009,5258269,2013-05-06,Concurrent,12.0,1461.0,State Facility,Possession With Intent to Deliver: Cocaine (< ...,NaN,6.0,NaN
1663884,1435009,5258269,2013-05-06,Concurrent,12.0,1461.0,State Facility,Possession With Intent to Deliver: Cocaine (< ...,NaN,6.0,NaN
1663885,1435009,5258269,2013-05-06,Concurrent,12.0,1461.0,State Facility,Possession With Intent to Deliver: Cocaine (< ...,NaN,6.0,NaN
1663886,1435009,5258269,2013-05-06,Consecutive,12.0,1461.0,State Facility,Possession With Intent to Deliver: Cocaine (< ...,NaN,6.0,NaN


### merge all new cols together

In [320]:
phl

,ID_VARIABLE,DOS,CTY_PHL
0,1000001,2010-02-18,0
1,1000002,2017-01-31,0
2,1000003,2002-05-08,0
3,1000003,2009-03-04,0
4,1000004,2013-12-10,0
...,...,...,...
1495977,1916193,2002-01-07,0
1495978,1916194,2016-11-14,0
1495979,1916195,2009-06-04,0
1495980,1916196,2014-03-03,0


In [321]:
charge_counts_grouped

,ID_VARIABLE,DOS,JPR_ID,OLD_DOS,CHARGE_COUNT,OGS,INC_REL_NUMERIC,JP_MIN
0,1000001,2010-02-18,4915383,2010-02-18,3,3.0,NaN,16.0
1,1000002,2017-01-31,5678165,2017-01-31,2,3.0,1.0,120.0
2,1000003,2002-05-08,1070248,2002-05-08,1,3.0,NaN,0.0
3,1000003,2009-03-04,4902797,2009-03-04,1,3.0,NaN,92.0
4,1000004,2013-12-10,5318013,2013-12-10,1,1.0,NaN,0.0
...,...,...,...,...,...,...,...,...
1495977,1916193,2002-01-07,425719,2002-01-07,1,5.0,NaN,0.0
1495978,1916194,2016-11-14,5557602,2016-11-14,1,5.0,NaN,0.0
1495979,1916195,2009-06-04,3884756,2009-06-04,1,5.0,NaN,0.0
1495980,1916196,2014-03-03,5351194,2014-03-03,1,5.0,NaN,31.0


In [322]:
print(len(phl), len(charge_counts_grouped))

1495982 1495982


In [323]:
supplemental_cols = pd.merge(phl.reset_index(), charge_counts_grouped[['ID_VARIABLE', 'DOS', 'CHARGE_COUNT', 'INC_REL_NUMERIC']], on=['ID_VARIABLE', 'DOS'])

In [324]:
supplemental_cols

,index,ID_VARIABLE,DOS,CTY_PHL,CHARGE_COUNT,INC_REL_NUMERIC
0,0,1000001,2010-02-18,0,3,NaN
1,1,1000002,2017-01-31,0,2,1.0
2,2,1000003,2002-05-08,0,1,NaN
3,3,1000003,2009-03-04,0,1,NaN
4,4,1000004,2013-12-10,0,1,NaN
...,...,...,...,...,...,...
1495977,1495977,1916193,2002-01-07,0,1,NaN
1495978,1495978,1916194,2016-11-14,0,1,NaN
1495979,1495979,1916195,2009-06-04,0,1,NaN
1495980,1495980,1916196,2014-03-03,0,1,NaN


In [325]:
supplemental_cols.to_csv('./supplemental_cols.csv', index=False)

In [ ]:
main_df[(main_df['ID_VARIABLE']==1232916) & (main_df['DOS']==pd.to_datetime('2016-04-26'))][['ID_VARIABLE', 'DOS', 'JPR_ID', 'JP_MIN', 'INCMIN', 'INC_RELATIONSHIP', 'OFN_LABEL', 'OGS']].sort_values(by='JPR_ID')

In [327]:
main_df[(main_df['ID_VARIABLE']==1232916) ][['ID_VARIABLE', 'DOS', 'JPR_ID', 'JP_MIN', 'INCMIN', 'INC_RELATIONSHIP', 'OFN_LABEL', 'OGS']].sort_values(by='JPR_ID')

,ID_VARIABLE,DOS,JPR_ID,JP_MIN,INCMIN,INC_RELATIONSHIP,OFN_LABEL,OFN_LABEL,OGS
997226,1232916,2008-10-13,2863528,7305.0,60.0,Consecutive,Kidnapping,NaN,9.0
997227,1232916,2008-10-13,2863528,7305.0,60.0,Consecutive,Burglary - Home: Person Present,NaN,9.0
997228,1232916,2008-10-13,2863528,7305.0,60.0,NaN,Kidnapping,NaN,10.0
997229,1232916,2008-10-13,2863528,7305.0,60.0,Consecutive,Robbery - Threatens S.B.I.,NaN,10.0


In [271]:
charge_counts_grouped[charge_counts_grouped['ID_VARIABLE']==1002106]

,ID_VARIABLE,DOS,JPR_ID,CHARGE_COUNT,OGS,INC_REL_NUMERIC,JP_MIN
3443,1002106,2014-01-28,5474679,6,10.0,1.0,4748.0
3444,1002106,2014-09-23,5402465,4,9.0,1.0,2769.0


In [273]:
main_df[(main_df['JPR_ID']==5402465)][['ID_VARIABLE', 'DOS']]

,ID_VARIABLE,DOS
1829712,1002106,2014-09-23
1829715,1002106,2014-09-23
1829718,1002106,2014-09-23
1829719,1002106,2014-09-23
2111997,1002106,2016-04-26
2111998,1002106,2016-04-26
2111999,1002106,2016-04-26
2112000,1002106,2016-04-26


In [274]:
main_df.groupby(by=['ID_VARIABLE', 'JPR_ID'])['DOS'].nunique().sort_values(ascending=False)

ID_VARIABLE  JPR_ID 
1884187      5199873    3
1692377      5087069    3
1819935      5299118    3
1528197      5051245    2
1130813      5350926    2
                       ..
1305594      271993     1
             271930     1
1305593      5431263    1
             2864596    1
1916197      5830496    1
Name: DOS, Length: 1581700, dtype: int64

In [277]:
main_df[(main_df['ID_VARIABLE']==1884187) & (main_df['JPR_ID']==5199873)][['ID_VARIABLE', 'JPR_ID', 'DOF', 'DOS', 'JP_MIN']].sort_values(by='DOS')

,ID_VARIABLE,JPR_ID,DOF,DOS,JP_MIN
1591799,1884187,5199873,01 Nov 11,2012-12-06,2556.0
1591800,1884187,5199873,01 Nov 11,2012-12-06,2556.0
1591801,1884187,5199873,01 Nov 11,2012-12-06,2556.0
1591802,1884187,5199873,01 Nov 11,2012-12-06,2556.0
1591803,1884187,5199873,01 Nov 11,2012-12-06,2556.0
1591804,1884187,5199873,01 Nov 11,2012-12-06,2556.0
1885851,1884187,5199873,01 Nov 11,2014-10-09,2557.0
1885850,1884187,5199873,01 Nov 11,2014-10-09,2557.0
1885849,1884187,5199873,01 Nov 11,2014-10-09,2557.0
1885848,1884187,5199873,01 Nov 11,2014-10-09,2557.0
